<h1> <div align="center"><a> Проект «Выбор локации для скважины» </a></div>

## Загрузка и подготовка данных

In [21]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error,r2_score
import numpy as np

In [22]:
df_0 = pd.read_csv("/datasets/geo_data_0.csv")
df_1 = pd.read_csv("/datasets/geo_data_1.csv")
df_2 = pd.read_csv("/datasets/geo_data_2.csv")

display(df_0.head(3), df_1.head(3), df_2.head(3))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


In [23]:
display(df_0.info())
display(df_1.info())
display(df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

Каждая таблица имеет 100 000 строк и 5 столбцов. Переименовывать столбцы нет необходимости. Пропущенных значений нет. В массиве представлены как категориальные (id), так и количественные признаки. Замена типов данных не требуется.
**Столбец id мы можем удалить, поскольку они не несет полезной нагрузки.**

In [24]:
df_0 = df_0.drop('id', axis=1)
df_1 = df_1.drop('id', axis=1)
df_2 = df_2.drop('id', axis=1)

In [25]:
#исследуем основные характеристики массивов
display(df_0.describe(), df_1.describe(), df_2.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Все три признака f0, f1, f2 расположены в диапазоне от -32 до 29, поэтому их нормализация и/или стандартизация не требуются.
Можно отметить, что наибольший средний запас сырья в третьем регионе (95 тыс. баррелей), а наименьшая средняя во втором (69 тыс.баррелей).

## Обучение и проверка модели

In [26]:
#объявим переменные с признаками и целевым признаком для каждой выборки
x_0 = df_0.drop(['product'], axis = 1)
y_0 = df_0['product']
x_1 = df_1.drop(['product'], axis = 1)
y_1 = df_1['product']
x_2 = df_2.drop(['product'], axis = 1)
y_2 = df_2['product']

In [27]:
#теперь разобьем данные на обучающую и валидационную выборки в соотношении 75:25.
x_0_train, x_0_valid, y_0_train, y_0_valid = train_test_split(x_0, y_0, test_size=0.25, random_state=12345)
x_1_train, x_1_valid, y_1_train, y_1_valid = train_test_split(x_1, y_1, test_size=0.25, random_state=12345)
x_2_train, x_2_valid, y_2_train, y_2_valid = train_test_split(x_2, y_2, test_size=0.25, random_state=12345)

In [28]:
#построим модель линейное регрессии для первого региона
lr = LinearRegression()
lr.fit(x_0_train, y_0_train)
predicted_valid_0 = lr.predict(x_0_valid)
mse = mean_squared_error(y_0_valid, predicted_valid_0)
r2 = lr.score(x_0_valid, y_0_valid)
print("Средний запас сырья для первого региона:", "{:,.2f}".format(predicted_valid_0.mean()))
print("RMSE =", "{:,.2f}".format(mse ** 0.5))
print("R2 на валидационной выборке:", "{:,.2f}".format(r2))

Средний запас сырья для первого региона: 92.59
RMSE = 37.58
R2 на валидационной выборке: 0.28


In [29]:
#построим модель для второго региона
lr = LinearRegression()
lr.fit(x_1_train,y_1_train)
predicted_valid_1 = lr.predict(x_1_valid)
mse = mean_squared_error(y_1_valid, predicted_valid_1)
r2 = lr.score(x_1_valid, y_1_valid)
print("Средний запас сырья для второго региона:", "{:,.2f}".format(predicted_valid_1.mean()))
print("RMSE =", "{:,.2f}".format(mse ** 0.5))
print("R2 на валидационной выборке:", "{:,.2f}".format(r2))

Средний запас сырья для второго региона: 68.73
RMSE = 0.89
R2 на валидационной выборке: 1.00


In [30]:
#построим модель для третьего региона
lr = LinearRegression()
lr.fit(x_2_train,y_2_train)
predicted_valid_2 = lr.predict(x_2_valid)
mse = mean_squared_error(y_2_valid, predicted_valid_2)
r2 = lr.score(x_2_valid, y_2_valid)
print("Средний запас сырья для третьего региона:", "{:,.2f}".format(predicted_valid_2.mean()))
print("RMSE =", "{:,.2f}".format(mse ** 0.5))
print("R2 на валидационной выборке:", "{:,.2f}".format(r2))

Средний запас сырья для третьего региона: 94.97
RMSE = 40.03
R2 на валидационной выборке: 0.21


Аналогично фактическим данным, наибольший средний запас сырья, предсказанный моделью, приходится на третий регион, а наименьший на второй. Однако стоит обратить на метрики RMSE и R2 по второму региону: первая меньше 1, а вторая равна 1. Это говорит о том, что модель линейной регрессии очень точно прогнозирует целевой признак во втором регионе, в то время как для первого и третьего регионов эти метрики значительно хуже.

In [31]:
##построим модель линейное регрессии для каждого региона
x_train = [x_0_train, x_1_train, x_2_train]
y_train = [y_0_train, y_1_train, y_2_train]
x_valid = [x_0_valid, x_1_valid, x_2_valid]
y_valid = [y_0_valid, y_1_valid, y_2_valid]
region = ["Показатели для первого региона", "Показатели для второго региона", "Показатели для третьего региона"]

def model(x_train, y_train, x_valid, y_valid):
    for index in range(3):
        lr = LinearRegression()
        lr.fit(x_train[index],y_train[index])
        predicted_valid = lr.predict(x_valid[index])
        mse = mean_squared_error(y_valid[index], predicted_valid)
        r2 = lr.score(x_valid[index], y_valid[index])
        print(region[index],
              "Средний запас сырья:", "{:,.2f}".format(predicted_valid.mean()),
              "RMSE =", "{:,.2f}".format(mse ** 0.5),
              "R2 на валидационной выборке:", "{:,.2f}".format(r2))        
        
model(x_train, y_train, x_valid, y_valid)

Показатели для первого региона Средний запас сырья: 92.59 RMSE = 37.58 R2 на валидационной выборке: 0.28
Показатели для второго региона Средний запас сырья: 68.73 RMSE = 0.89 R2 на валидационной выборке: 1.00
Показатели для третьего региона Средний запас сырья: 94.97 RMSE = 40.03 R2 на валидационной выборке: 0.21


## Подготовка к расчёту прибыли

In [32]:
#сохраним все ключевые значения для расчётов в отдельных переменных
BUDGET = 10000000  # бюджет на разработку скважин в регионе 10 млрд рублей, но поскольку объём запасов указан в тысячах баррелей,
#то и бюджет переведем в тысяч рублей
PRICE = 450  # цена за один баррель сырья
RESEARCH = 500  # количество скважин для геолого-разведки
QUANTITY = 200  # количество скважин для разработки

In [33]:
#рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и всего региона в целом
pbp = BUDGET/(QUANTITY * PRICE)
print("Достаточный объем сырья для разработки новой скважины:","{:,.2f}".format(pbp), "\n"
      "Достаточный объем сырья для безубыточной разработки 200 скважин в регионе:", "{:,.2f}".format(pbp*QUANTITY), "\n"
      "Средний запас сырья для первого региона:", "{:,.2f}".format(y_0_valid.mean()), "\n"
      "Средний запас сырья для второго региона:", "{:,.2f}".format(y_1_valid.mean()), "\n"
      "Средний запас сырья для третьего региона:", "{:,.2f}".format(y_2_valid.mean()))

Достаточный объем сырья для разработки новой скважины: 111.11 
Достаточный объем сырья для безубыточной разработки 200 скважин в регионе: 22,222.22 
Средний запас сырья для первого региона: 92.08 
Средний запас сырья для второго региона: 68.72 
Средний запас сырья для третьего региона: 94.88


Для достижения точки безубыточности нам необходимо иметь средний запас сырья не менее 111.11 тыс. баррелей на одну скважину. Мы видим, что фактические средние запасы во всех трех регионах ниже данного значения. Таким образом, для каждого региона нам нужно отобрать 200 скважин с максимальными запасами, чтобы их общий запас был не меньше 22 222 тыс. баррелей. Потенциально это возможно во всех трех регионах, поскольку максимальные значения запасов превышают порог в 111 тыс. И поскольку на данном этапе ни один регион не показал достаточно хороших результатов, продолжим анализировать их все.

## Расчёт прибыли и рисков 

In [34]:
#cохраним значения predictions_valid в series с индексами из target_valid
predicted_valid_0 = pd.Series(predicted_valid_0, index = y_0_valid.index)
predicted_valid_1 = pd.Series(predicted_valid_1, index = y_1_valid.index)
predicted_valid_2 = pd.Series(predicted_valid_2, index = y_2_valid.index)

#теперь оценим реальные запасы сырья по лучшим 200 предсказанным скважинам
prob_sorted_0 = predicted_valid_0.sort_values(ascending = False)
top_200_0 = y_0_valid[prob_sorted_0.index][:QUANTITY].sum()
prob_sorted_1 = predicted_valid_1.sort_values(ascending = False)
top_200_1 = y_1_valid[prob_sorted_1.index][:QUANTITY].sum()
prob_sorted_2 = predicted_valid_2.sort_values(ascending = False)
top_200_2 = y_2_valid[prob_sorted_2.index][:QUANTITY].sum()

print("Запас сырья для 200 лучших скважин первого региона:", "{:,.2f}".format(top_200_0))
print("Запас сырья для 200 лучших скважин второго региона:", "{:,.2f}".format(top_200_1))
print("Запас сырья для 200 лучших скважин третьего региона:", "{:,.2f}".format(top_200_2))

Запас сырья для 200 лучших скважин первого региона: 29,601.84
Запас сырья для 200 лучших скважин второго региона: 27,589.08
Запас сырья для 200 лучших скважин третьего региона: 28,245.22


Как мы видим, объем запасов лучших 200 скважин во всех трех регионах превышает необходимый порог в 22 222 тыс.баррелей., что говорит о потенциальной прибыли в каждом из них. Напишем функцию, которая **по предсказаниям модели отберет 200 самых продуктивных точек в регионе и посчитает прибыль по данным из объема реальных скважин**.

In [35]:
def profit(y_valid, predicted_valid):
    prob_sorted = predicted_valid.sort_values(ascending=False)
    selected = y_valid[prob_sorted.index][:QUANTITY]
    return selected.sum() * PRICE - BUDGET

profit_0 = profit(y_0_valid, predicted_valid_0)
profit_1 = profit(y_1_valid, predicted_valid_1)
profit_2 = profit(y_2_valid, predicted_valid_2)
print("{:,.2f}".format(profit_0).replace(',', ' '))
print("{:,.2f}".format(profit_1).replace(',', ' '))
print("{:,.2f}".format(profit_2).replace(',', ' '))

3 320 826.04
2 415 086.70
2 710 349.96


In [42]:
#посчитаем максимальную прибыль для каждого региона
predicted_valid = [predicted_valid_0, predicted_valid_1, predicted_valid_2]
y_valid = [y_0_valid, y_1_valid, y_2_valid]
region = ["Показатели для первого региона", "Показатели для второго региона", "Показатели для третьего региона"]

def prof(y_valid, predicted_valid):
    for index in range(3):
        prob_sorted = predicted_valid[index].sort_values(ascending=False)
        selected = y_valid[index][prob_sorted.index][:QUANTITY]
        result = selected.sum() * PRICE - BUDGET
        print(region[index], "Максимальная прибыль:", "{:,.2f}".format(result))

prof(y_valid, predicted_valid)

Показатели для первого региона Максимальная прибыль: 3,320,826.04
Показатели для второго региона Максимальная прибыль: 2,415,086.70
Показатели для третьего региона Максимальная прибыль: 2,710,349.96


Все три региона потенциально прибыльные, однако максимальную потенциальную прибыль продемонстрировал первый регион. Применим технику bootstrap с 1000 выборок по 500 строк каждая (количество скважин для геолого-разведки), из которых в разработку пойдут 200 с наибольшими запасами, чтобы найти распределение прибыли в 95% доверительном интервале и оценить уровень риска.

In [37]:
state = np.random.RandomState(12345)
profit_0 = []
loss_0 = 0
for i in range(1000):
    x_subsample = predicted_valid_0.sample(n = RESEARCH, replace=True, random_state=state)
    y_subsample = y_0_valid[x_subsample.index]
    profit_0.append(profit(y_subsample, x_subsample))
    if profit(y_subsample, x_subsample) < 0:
        loss_0 += 1

profit_0 = pd.Series(profit_0)
profit_0_mean = profit_0.mean()
lower_0 = profit_0.quantile(0.025)
upper_0 = profit_0.quantile(0.975)
print("Показатели для первого региона")
print("Средняя выручка:", "{:,.2f}".format(profit_0_mean).replace(',', ' '))
print("Нижняя граница диапазона прогнозной прибыли:", "{:,.2f}".format(lower_0).replace(',', ' '))
print("Верхняя граница диапазона прогнозной прибыли:", "{:,.2f}".format(upper_0).replace(',', ' '))
print("Вероятность убытков:", "{:.2%}".format(loss_0/1000))

Показатели для первого региона
Средняя выручка: 425 938.53
Нижняя граница диапазона прогнозной прибыли: -102 090.09
Верхняя граница диапазона прогнозной прибыли: 947 976.35
Вероятность убытков: 6.00%


In [38]:
state = np.random.RandomState(12345)
profit_1 = []
loss_1 = 0
for i in range(1000):
    x_subsample = predicted_valid_1.sample(n = RESEARCH, replace=True, random_state=state)
    y_subsample = y_1_valid[x_subsample.index]
    profit_1.append(profit(y_subsample, x_subsample))
    if profit(y_subsample, x_subsample) < 0:
        loss_1 += 1

profit_1 = pd.Series(profit_1)
profit_1_mean = profit_1.mean()
lower_1 = profit_1.quantile(0.025)
upper_1 = profit_1.quantile(0.975)
print("Показатели для второго региона")
print("Средняя выручка:", "{:,.2f}".format(profit_1_mean).replace(',', ' '))
print("Нижняя граница диапазона прогнозной прибыли:", "{:,.2f}".format(lower_1).replace(',', ' '))
print("Верхняя граница диапазона прогнозной прибыли:", "{:,.2f}".format(upper_1).replace(',', ' '))
print("Вероятность убытков:", "{:.2%}".format(loss_1/1000))

Показатели для второго региона
Средняя выручка: 515 222.77
Нижняя граница диапазона прогнозной прибыли: 68 873.23
Верхняя граница диапазона прогнозной прибыли: 931 547.59
Вероятность убытков: 1.00%


In [39]:
state = np.random.RandomState(12345)
profit_2 = []
loss_2 = 0
for i in range(1000):
    x_subsample = predicted_valid_2.sample(n = RESEARCH, replace=True, random_state=state)
    y_subsample = y_2_valid[x_subsample.index]
    profit_2.append(profit(y_subsample, x_subsample))
    if profit(y_subsample, x_subsample) < 0:
        loss_2 += 1

profit_2 = pd.Series(profit_2)       
profit_2_mean = profit_2.mean()
lower_2 = profit_2.quantile(0.025)
upper_2 = profit_2.quantile(0.975)
print("Показатели для третьего региона")
print("Средняя выручка:", "{:,.2f}".format(profit_2_mean).replace(',', ' '))
print("Нижняя граница 95% доверительного интервала:", "{:,.2f}".format(lower_2).replace(',', ' '))
print("Верхняя граница 95% доверительного интервала", "{:,.2f}".format(upper_2).replace(',', ' '))
print("Вероятность убытков:", "{:.2%}".format(loss_2/1000))

Показатели для третьего региона
Средняя выручка: 435 008.36
Нижняя граница 95% доверительного интервала: -128 880.55
Верхняя граница 95% доверительного интервала 969 706.95
Вероятность убытков: 6.40%


In [40]:
#bootstrap также можно вложить в цикл и функцию
predicted_valid = [predicted_valid_0, predicted_valid_1, predicted_valid_2]
y_valid = [y_0_valid, y_1_valid, y_2_valid]
region = ["Показатели для первого региона", "Показатели для второго региона", "Показатели для третьего региона"]

def bootstrap(predicted_valid, y_valid):
    for index in range(3):
        state = np.random.RandomState(12345)
        prof = []
        for i in range(1000):
            x_subsample = predicted_valid[index].sample(n = RESEARCH, replace=True, random_state=state)
            y_subsample = y_valid[index][x_subsample.index]
            prof.append(profit(y_subsample, x_subsample))
            
        prof = pd.Series(prof)
        prof_mean = prof.mean()
        lower = prof.quantile(0.025)
        upper = prof.quantile(0.975)
        print(region[index], "\n"
              "Средняя выручка:", "{:,.2f}".format(prof_mean).replace(',', ' '), "\n"
              "Нижняя граница 95% диапазона прогнозной прибыли:", "{:,.2f}".format(lower).replace(',', ' '), "\n"
              "Верхняя граница 95% диапазона прогнозной прибыли:", "{:,.2f}".format(upper).replace(',', ' '), "\n"
              "Вероятность убытков:", "{:.2%}".format((prof < 0).mean()), "\n"
              ""
             )
        
bootstrap(predicted_valid, y_valid)

Показатели для первого региона 
Средняя выручка: 425 938.53 
Нижняя граница 95% диапазона прогнозной прибыли: -102 090.09 
Верхняя граница 95% диапазона прогнозной прибыли: 947 976.35 
Вероятность убытков: 6.00% 

Показатели для второго региона 
Средняя выручка: 515 222.77 
Нижняя граница 95% диапазона прогнозной прибыли: 68 873.23 
Верхняя граница 95% диапазона прогнозной прибыли: 931 547.59 
Вероятность убытков: 1.00% 

Показатели для третьего региона 
Средняя выручка: 435 008.36 
Нижняя граница 95% диапазона прогнозной прибыли: -128 880.55 
Верхняя граница 95% диапазона прогнозной прибыли: 969 706.95 
Вероятность убытков: 6.40% 



Для каждого региона с помощью метода bootstrap мы определили среднюю выручку, диапазон 95% доверительного интервала, а также риск убытков. Наибольшую среднюю прибыль продемонстрировал второй регион. Однако помимо прибыли необходимо принимать во внимание риск возникноваения убытков. Приемлемый уровень риска убытков установлен на уровне 2.5%. Этому показателю удовлетворяет также только второй регион. В связи с этим **можно рекомендовать для разработки скважин второй регион, который имеет как наибольшее значение средней выручки (515 млн.руб.), так и наименьший уровень риска (1%).** 